## Table V: Accuracy and Speedup on Image VLMs
### Preparation
Change the `res_path` variable to the path of simulator results. 
Change the `acc_path` variable to the path of algorithm evaluation results

In [3]:
res_path = '../../../simulator/example_sim_results/'
acc_path = '../../../algorithm/example_output/'

Investigate how Focus generalize beyond Video-input VLMs. Compare Focus with baselines in image-input VLMs

In [4]:
import pandas as pd
import numpy as np

model_display = {'llava_onevision': 'Llava-OV', 'qwen2_5_vl': 'Qwen2.5-VL'}
dataset_display = {'vqav2': 'VQAv2', 'mme': 'MME', 'mmbench': 'MMBench'}

df_dense = pd.read_csv(res_path + '/main_dense.csv')
df_adaptiv = pd.read_csv(res_path + '/main_adaptiv.csv')
df_focus = pd.read_csv(res_path + '/main_focus.csv')
df_acc = pd.read_csv(acc_path + '/accuracy.csv')

df_dense = df_dense[df_dense['model'].isin(model_display.keys()) & 
                     df_dense['dataset'].isin(dataset_display.keys())]
df_adaptiv = df_adaptiv[df_adaptiv['model'].isin(model_display.keys()) & 
                         df_adaptiv['dataset'].isin(dataset_display.keys())]
df_focus = df_focus[df_focus['model'].isin(model_display.keys()) & 
                     df_focus['dataset'].isin(dataset_display.keys())]

result = []

for model_key in model_display.keys():
    for dataset_key in dataset_display.keys():
        dense_match = df_dense[(df_dense['model'] == model_key) & (df_dense['dataset'] == dataset_key)]
        adaptiv_match = df_adaptiv[(df_adaptiv['model'] == model_key) & (df_adaptiv['dataset'] == dataset_key)]
        focus_match = df_focus[(df_focus['model'] == model_key) & (df_focus['dataset'] == dataset_key)]
        
        if dense_match.empty:
            continue
        
        dense_time = dense_match.iloc[0]['execution_time']
        adaptiv_time = adaptiv_match.iloc[0]['execution_time'] if not adaptiv_match.empty else np.nan
        focus_time = focus_match.iloc[0]['execution_time'] if not focus_match.empty else np.nan
        
        dense_speedup = 1.00
        adaptiv_speedup = dense_time / adaptiv_time if not np.isnan(adaptiv_time) else np.nan
        focus_speedup = dense_time / focus_time if not np.isnan(focus_time) else np.nan
        
        acc_match = df_acc[(df_acc['Models'] == model_key) & 
                            (df_acc['Dataset'] == dataset_key)]
        
        if not acc_match.empty:
            dense_acc = acc_match.iloc[0]['Dense']
            adaptiv_acc = acc_match.iloc[0]['Adaptiv'] if not pd.isna(acc_match.iloc[0]['Adaptiv']) else np.nan
            focus_acc = acc_match.iloc[0]['Focus']
        else:
            print(f"Warning: No accuracy data for {model_key} - {dataset_key}")
            dense_acc = adaptiv_acc = focus_acc = np.nan
        
        metric = 'Score' if dataset_key == 'mme' else 'Accuracy'
        speedup_row = {
            'Models': model_display[model_key],
            'Dataset': dataset_display[dataset_key],
            'Metric': 'Speedup',
            'Dense': f'{dense_speedup:.2f}',
            'AdapTiV': f'{adaptiv_speedup:.2f}' if not np.isnan(adaptiv_speedup) else 'N/A',
            'Ours': f'{focus_speedup:.2f}' if not np.isnan(focus_speedup) else 'N/A'
        }
        
        acc_row = {
            'Models': '',
            'Dataset': '',
            'Metric': metric,
            'Dense': f'{dense_acc:.2f}' if not np.isnan(dense_acc) else 'N/A',
            'AdapTiV': f'{adaptiv_acc:.2f}' if not np.isnan(adaptiv_acc) else 'N/A',
            'Ours': f'{focus_acc:.2f}' if not np.isnan(focus_acc) else 'N/A'
        }
        
        result.append(speedup_row)
        result.append(acc_row)

df_result = pd.DataFrame(result)

print("="*100)
print("TABLE V: ACCURACY AND SPEEDUP ON IMAGE VLMs")
print("="*100)
print(df_result.to_string(index=False))
print("="*100)

TABLE V: ACCURACY AND SPEEDUP ON IMAGE VLMs
    Models Dataset   Metric   Dense AdapTiV    Ours
  Llava-OV   VQAv2  Speedup    1.00    5.19    4.44
                   Accuracy   84.32   82.48   83.01
  Llava-OV     MME  Speedup    1.00    1.65    4.26
                      Score 1067.27 1036.27 1044.79
  Llava-OV MMBench  Speedup    1.00    1.60    4.25
                   Accuracy   84.99   84.49   83.46
Qwen2.5-VL   VQAv2  Speedup    1.00    1.96    1.91
                   Accuracy   84.48   79.77   81.73
Qwen2.5-VL     MME  Speedup    1.00    1.89    1.97
                      Score 1337.66 1129.56 1238.88
Qwen2.5-VL MMBench  Speedup    1.00    1.93    1.78
                   Accuracy   85.69   83.79   84.46
